In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

np.random.seed(45)
num_samples = 40
    
# Generate data
x1 = np.random.uniform(-1, 1, num_samples)
f_x = 3*x1 + 4
eps = np.random.randn(num_samples)
y = f_x + eps

In [2]:
def reg(x,theta_0,theta_1):
    return theta_1*x + theta_0
def mse(y_hat,y):
    return torch.mean((y_hat-y)**2)

In [3]:
x1_torch = torch.from_numpy(x1)
y_torch = torch.from_numpy(y)
theta_0 = torch.tensor(1.0,requires_grad=True)
theta_1 = torch.tensor(0.0, requires_grad=True)

y_hat = reg(x1_torch,theta_0,theta_1)
loss = mse(y_hat,y_torch)
loss.backward()
theta_0_true_grad = theta_0.grad.item()
theta_1_true_grad = theta_1.grad.item()
print('The True gradient values of theta_0, theta_1:',theta_0_true_grad,theta_1_true_grad)

The True gradient values of theta_0, theta_1: -5.447053909301758 -1.1946825981140137


In [4]:
t0_stochastic_gradients = []
t1_stochastic_gradients = []
for i in range(num_samples):
    theta_0.grad = None
    theta_1.grad = None
    y_hat = reg(x1_torch[i],theta_0,theta_1)
    loss = mse(y_hat,y_torch[i])
    loss.backward()
    t0_stochastic_gradients.append(theta_0.grad.item())
    t1_stochastic_gradients.append(theta_1.grad.item())

t0_avg_grad = sum(t0_stochastic_gradients) / num_samples
t1_avg_grad = sum(t1_stochastic_gradients)/num_samples
print('The True gradient values of theta_0, theta_1:',theta_0_true_grad,theta_1_true_grad)
print('The average value of stochastic gradient of theta_0, theta_1 for all points in the dataset:',t0_avg_grad,t1_avg_grad)

The True gradient values of theta_0, theta_1: -5.447053909301758 -1.1946825981140137
The average value of stochastic gradient of theta_0, theta_1 for all points in the dataset: -5.447053959220648 -1.194682613387704


In [5]:
print('The difference between avg of stochastic and true grad is',abs(theta_0_true_grad-t0_avg_grad),abs(theta_1_true_grad-t1_avg_grad))

The difference between avg of stochastic and true grad is 4.9918890354661016e-08 1.527369031251169e-08


In [ ]:
def full_batch_gd(x,y,lr = 0.01,epsilon = 0.001):
    theta_0 = torch.tensor(0.0,requires_grad=True)
    theta_1 = torch.tensor(0.0,requires_grad=True)
    t0_hist = [theta_0.item()]
    t1_hist = [theta_1.item()]
    losses = []
    while loss>=epsilon:
        theta_0.grad.zero_()
        theta_1.grad.zero_()
        y_hat = reg(x,theta_0,theta_1)
        loss = mse(y_hat,y)
        losses.append(loss)
        loss.backward()
        with torch.no_grad():
            theta_0 = theta_0-lr*theta_0.grad
            theta_1 = theta_1-lr*theta_1.grad

        t0_hist.append(theta_0.grad.item())
        t1_hist.append(theta_1.grad.item())

    return losses, theta_0.grad.item(), theta_1.grad.item(),t0_hist,t1_hist
def mini_batch_gd(x,y,lr = 0.01,batch_size = 2,epochs = 15,epsilon = 0.001):
    theta_0 = torch.tensor(0.0,requires_grad=True)
    theta_1 = torch.tensor(0.0,requires_grad=True)
    t0_hist = [theta_0.item()]
    t1_hist = [theta_1.item()]
    losses = []